# OpenAI APIs - Vision

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/vision).
This tutorial covers the vision APIs for vision language models.

SGLang supports vision language models such as Llama 3.2, LLaVA-OneVision, and QWen-VL2  
- [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)  
- [lmms-lab/llava-onevision-qwen2-72b-ov-chat](https://huggingface.co/lmms-lab/llava-onevision-qwen2-72b-ov-chat)  
- [Qwen/Qwen2-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2-VL-7B-Instruct)  

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

Remember to add `--chat-template llama_3_vision` to specify the vision chat template, otherwise the server only supports text.
We need to specify `--chat-template` for vision language models because the chat template provided in Hugging Face tokenizer only supports text.

In [1]:
from sglang.utils import (
    execute_shell_command,
    wait_for_server,
    terminate_process,
    print_highlight,
)

embedding_process = execute_shell_command(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-11B-Vision-Instruct \
    --port=30000 --chat-template=llama_3_vision
"""
)

wait_for_server("http://localhost:30000")

[2025-01-28 02:24:31] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_mode='auto', load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization_param_path=None, quantization=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-11B-Vision-Instruct', chat_template='llama_3_vision', is_embedding=False, revision=None, skip_tokenizer_init=False, host='127.0.0.1', port=30000, mem_fraction_static=0.88, max_running_requests=None, max_total_tokens=None, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='lpm', schedule_conservativeness=1.0, cpu_offload_gb=0, prefill_only_one_req=False, tp_size=1, stream_interval=1, stream_output=False, random_seed=375031431, constrained_json_whitespace_pattern=None, watchdog_timeout=300, download_dir=None, base_gpu_id=0, log_level='info', log_level_http=None, log_requests=False, show_

[2025-01-28 02:24:38] Use chat template for the OpenAI-compatible API server: llama_3_vision


[2025-01-28 02:24:51 TP0] Overlap scheduler is disabled for multimodal models.
[2025-01-28 02:24:51 TP0] Automatically reduce --mem-fraction-static to 0.836 because this is a multimodal model.
[2025-01-28 02:24:51 TP0] Automatically turn off --chunked-prefill-size for mllama.
[2025-01-28 02:24:51 TP0] Init torch distributed begin.


[2025-01-28 02:24:52 TP0] Load weight begin. avail mem=78.81 GB


[2025-01-28 02:24:53 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


...

100%|██████████| 23/23 [00:06<00:00,  3.47it/s]
[2025-01-28 02:25:04 TP0] Capture cuda graph end. Time elapsed: 6.64 s


[2025-01-28 02:25:06 TP0] max_total_num_tokens=299646, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=2049, context_len=131072
[2025-01-28 02:25:06] INFO:     Started server process [2855121]
[2025-01-28 02:25:06] INFO:     Waiting for application startup.
[2025-01-28 02:25:06] INFO:     Application startup complete.
[2025-01-28 02:25:06] INFO:     Uvicorn running on http://127.0.0.1:30000 (Press CTRL+C to quit)


[2025-01-28 02:25:07] INFO:     127.0.0.1:42562 - "GET /v1/models HTTP/1.1" 200 OK


[2025-01-28 02:25:07] INFO:     127.0.0.1:42574 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-01-28 02:25:07 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-28 02:25:09] INFO:     127.0.0.1:42578 - "POST /generate HTTP/1.1" 200 OK
[2025-01-28 02:25:09] The server is fired up and ready to roll!


## Using cURL

Once the server is up, you can send test requests using curl or requests.

In [2]:
import subprocess

curl_command = """
curl -s http://localhost:30000/v1/chat/completions \
  -d '{
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "What’s in this image?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }'
"""

response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)

[2025-01-28 02:25:16 TP0] Prefill batch. #new-seq: 1, #new-token: 6463, #cached-token: 0, cache hit rate: 0.00%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-28 02:25:16] INFO:     127.0.0.1:51754 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = "http://localhost:30000/v1/chat/completions"

data = {
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    "max_tokens": 300,
}

response = requests.post(url, json=data)
print_highlight(response.text)

[2025-01-28 02:25:17 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 6462, cache hit rate: 49.97%, token usage: 0.02, #running-req: 0, #queue-req: 0
[2025-01-28 02:25:17 TP0] Decode batch. #running-req: 1, #token: 6467, token usage: 0.02, gen throughput (token/s): 3.69, #queue-req: 0


[2025-01-28 02:25:17 TP0] Decode batch. #running-req: 1, #token: 6507, token usage: 0.02, gen throughput (token/s): 89.38, #queue-req: 0


[2025-01-28 02:25:18 TP0] Decode batch. #running-req: 1, #token: 6547, token usage: 0.02, gen throughput (token/s): 84.20, #queue-req: 0


[2025-01-28 02:25:18 TP0] Decode batch. #running-req: 1, #token: 6587, token usage: 0.02, gen throughput (token/s): 91.86, #queue-req: 0
[2025-01-28 02:25:18] INFO:     127.0.0.1:51760 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)

[2025-01-28 02:25:20 TP0] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 6452, cache hit rate: 66.58%, token usage: 0.02, #running-req: 0, #queue-req: 0


[2025-01-28 02:25:20 TP0] Decode batch. #running-req: 1, #token: 6501, token usage: 0.02, gen throughput (token/s): 23.49, #queue-req: 0


[2025-01-28 02:25:20 TP0] Decode batch. #running-req: 1, #token: 6541, token usage: 0.02, gen throughput (token/s): 107.12, #queue-req: 0


[2025-01-28 02:25:21 TP0] Decode batch. #running-req: 1, #token: 6581, token usage: 0.02, gen throughput (token/s): 106.71, #queue-req: 0


[2025-01-28 02:25:21 TP0] Decode batch. #running-req: 1, #token: 6621, token usage: 0.02, gen throughput (token/s): 106.69, #queue-req: 0
[2025-01-28 02:25:21] INFO:     127.0.0.1:51774 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Multiple-Image Inputs

The server also supports multiple images and interleaved text and images if the model supports it.

In [5]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:30000/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ],
    temperature=0,
)

print_highlight(response.choices[0].message.content)

[2025-01-28 02:25:22 TP0] Prefill batch. #new-seq: 1, #new-token: 12895, #cached-token: 0, cache hit rate: 39.99%, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-01-28 02:25:23 TP0] Decode batch. #running-req: 1, #token: 12931, token usage: 0.04, gen throughput (token/s): 27.44, #queue-req: 0


[2025-01-28 02:25:23] INFO:     127.0.0.1:44510 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(embedding_process)

## Chat Template

As mentioned before, if you do not specify a vision model's `--chat-template`, the server uses Hugging Face's default template, which only supports text.

We list popular vision models with their chat templates:

- [meta-llama/Llama-3.2-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct) uses `llama_3_vision`.
- [Qwen/Qwen2-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2-VL-7B-Instruct) uses `qwen2-vl`.
- [LlaVA-OneVision](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov) uses `chatml-llava`.
- [LLaVA-NeXT](https://huggingface.co/collections/lmms-lab/llava-next-6623288e2d61edba3ddbf5ff) uses `chatml-llava`.
- [Llama3-LLaVA-NeXT](https://huggingface.co/lmms-lab/llama3-llava-next-8b) uses `llava_llama_3`.
- [LLaVA-v1.5 / 1.6](https://huggingface.co/liuhaotian/llava-v1.6-34b) uses `vicuna_v1.1`.